<a href="https://colab.research.google.com/github/robert-myers/myanimelist-recommender/blob/master/notesbooks/try_it_out.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [145]:
pip install jikanpy && pip install surprise

     |████████████████████████████████| 1.2MB 4.1MB/s 
     |████████████████████████████████| 153kB 19.7MB/s 
     |████████████████████████████████| 256kB 20.8MB/s 
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-none-any.whl size=3162 sha256=3d08797215393b599659cda182d9d123c123879e76318cc09e4b4efc2cfb9251
  Stored in directory: /root/.cache/pip/wheels/d3/00/b3/32d613e19e08a739751dd6bf998cfed277728f8b2127ad4eb7
Successfully built idna-ssl


In [0]:
import pandas as pd
import time

from jikanpy import Jikan
from surprise import accuracy
from surprise.dump import load

In [0]:
jikan = Jikan()

In [0]:
users = pd.read_csv("https://s3.us-east-2.amazonaws.com/my.anime.list.sagemaker/myanimelist/UserList.csv")
users = users[["username"]]
users = users.dropna()
users["lower"] = users["username"].apply(lambda x: x.lower())
users = users.set_index("lower")

In [0]:
users = dict(users.T)

In [0]:
def get_uid(username):
  username = username.lower()
  if username in users:
    return users[username.lower()][0]
  else:
    return username

In [123]:
anime = pd.read_csv("https://s3.us-east-2.amazonaws.com/my.anime.list.sagemaker/myanimelist/AnimeList.csv", index_col="anime_id")[["title",	"image_url"]]
anime.reset_index()[["anime_id"]]

,anime_id
0,11013
1,2104
2,5262
3,721
4,12365
...,...
14473,26089
14474,21525
14475,37897
14476,34193


In [0]:
surprised = load("/content/drive/My Drive/svd.pickle")

In [0]:
algo = surprised[1]
predictions = surprised[0]

In [8]:
accuracy.rmse(predictions)

RMSE: 1.1859


1.1859182533000143

In [0]:
%%time
orionite_completed = pd.DataFrame(jikan.user(username=user, request='animelist', argument='completed', page=1)["anime"])
i = 2
downloading = True
while downloading:
  time.sleep(4)
  downloading = jikan.user(username=user, request='animelist', argument='completed', page=i)["anime"]
  orionite_completed = orionite_completed.append(pd.DataFrame(downloading))
  i += 1

In [264]:
orionite_completed

,mal_id,title,video_url,url,image_url,type,watching_status,score,watched_episodes,total_episodes,airing_status,season_name,season_year,has_episode_video,has_promo_video,has_video,is_rewatching,tags,rating,start_date,end_date,watch_start_date,watch_end_date,days,storage,priority,added_to_list,studios,licensors
0,11759,Accel World,https://myanimelist.net/anime/11759/Accel_Worl...,https://myanimelist.net/anime/11759/Accel_World,https://cdn.myanimelist.net/images/anime/8/381...,TV,2,0,24,24,2,None,None,False,True,True,False,None,PG-13,2012-07-04T00:00:00+00:00,2013-10-09T00:00:00+00:00,2013-03-04T00:00:00+00:00,2013-10-09T00:00:00+00:00,161.0,None,Low,False,[],[]
1,1292,Afro Samurai,https://myanimelist.net/anime/1292/Afro_Samura...,https://myanimelist.net/anime/1292/Afro_Samurai,https://cdn.myanimelist.net/images/anime/3/158...,TV,2,8,5,5,2,None,None,False,True,True,False,None,R+,2007-03-05T00:00:00+00:00,2007-04-05T00:00:00+00:00,None,None,NaN,None,Low,False,[],[]
2,366,AIKa,https://myanimelist.net/anime/366/AIKa/video,https://myanimelist.net/anime/366/AIKa,https://cdn.myanimelist.net/images/anime/11/61...,OVA,2,7,7,7,2,None,None,False,True,True,False,None,R+,1999-01-04T00:00:00+00:00,2001-01-04T00:00:00+00:00,2008-02-07T00:00:00+00:00,2008-05-07T00:00:00+00:00,4.0,None,Low,False,[],[]
3,2163,AIKa R-16: Virgin Mission,https://myanimelist.net/anime/2163/AIKa_R-16__...,https://myanimelist.net/anime/2163/AIKa_R-16__...,https://cdn.myanimelist.net/images/anime/3/906...,OVA,2,8,3,3,2,None,None,False,False,False,False,None,R+,2009-01-04T00:00:00+00:00,2009-02-10T00:00:00+00:00,2008-07-07T00:00:00+00:00,2008-09-07T00:00:00+00:00,3.0,None,Low,False,[],[]
4,4718,AIKa: Special Trial,https://myanimelist.net/anime/4718/AIKa__Speci...,https://myanimelist.net/anime/4718/AIKa__Speci...,https://cdn.myanimelist.net/images/anime/3/824...,Special,2,7,1,1,2,None,None,False,False,False,False,None,R+,2000-01-05T00:00:00+00:00,2000-01-05T00:00:00+00:00,2009-03-07T00:00:00+00:00,2009-03-07T00:00:00+00:00,1.0,None,Low,False,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,7745,Zettai Karen Children OVA: Aitazousei! Ubaware...,https://myanimelist.net/anime/7745/Zettai_Kare...,https://myanimelist.net/anime/7745/Zettai_Kare...,https://cdn.myanimelist.net/images/anime/9/252...,OVA,2,8,1,1,2,None,None,False,True,True,False,None,PG-13,2011-04-07T00:00:00+00:00,2011-04-07T00:00:00+00:00,2010-05-08T00:00:00+00:00,2010-05-08T00:00:00+00:00,1.0,None,Low,False,[],[]
150,763,Zoids,https://myanimelist.net/anime/763/Zoids/video,https://myanimelist.net/anime/763/Zoids,https://cdn.myanimelist.net/images/anime/2/154...,TV,2,8,67,67,2,None,None,False,False,False,False,None,PG,1999-04-09T00:00:00+00:00,2001-11-12T00:00:00+00:00,None,None,NaN,None,Low,False,[],[]
151,764,Zoids Shinseiki/Zero,https://myanimelist.net/anime/764/Zoids_Shinse...,https://myanimelist.net/anime/764/Zoids_Shinse...,https://cdn.myanimelist.net/images/anime/11/24...,TV,2,7,26,26,2,None,None,False,True,True,False,None,PG,2001-06-01T00:00:00+00:00,2003-06-06T00:00:00+00:00,None,None,NaN,None,Low,False,[],[]
152,3228,Zoku Sayonara Zetsubou Sensei,https://myanimelist.net/anime/3228/Zoku_Sayona...,https://myanimelist.net/anime/3228/Zoku_Sayona...,https://cdn.myanimelist.net/images/anime/13/75...,TV,2,8,13,13,2,None,None,False,False,False,False,None,R,2008-06-01T00:00:00+00:00,2010-06-03T00:00:00+00:00,2008-12-01T00:00:00+00:00,2008-12-06T00:00:00+00:00,153.0,None,Low,False,[],[]


In [260]:
jikan.user(username=user, request='animelist', argument='completed', page=3)["anime"])

ValueError: ignored

In [245]:
orionite_completed[orionite_completed["title"].str.contains("Tengen")]

,mal_id,title,video_url,url,image_url,type,watching_status,score,watched_episodes,total_episodes,airing_status,season_name,season_year,has_episode_video,has_promo_video,has_video,is_rewatching,tags,rating,start_date,end_date,watch_start_date,watch_end_date,days,storage,priority,added_to_list,studios,licensors
107,2001,Tengen Toppa Gurren Lagann,https://myanimelist.net/anime/2001/Tengen_Topp...,https://myanimelist.net/anime/2001/Tengen_Topp...,https://cdn.myanimelist.net/images/anime/4/512...,TV,2,10,27,27,2,None,None,False,True,True,False,None,PG-13,2007-01-04T00:00:00+00:00,2009-06-09T00:00:00+00:00,2009-01-06T00:00:00+00:00,2009-06-09T00:00:00+00:00,98.0,None,Low,False,[],[]
108,4107,Tengen Toppa Gurren Lagann Movie 1: Gurren-hen,https://myanimelist.net/anime/4107/Tengen_Topp...,https://myanimelist.net/anime/4107/Tengen_Topp...,https://cdn.myanimelist.net/images/anime/10/11...,Movie,2,9,1,1,2,None,None,False,True,True,False,None,PG-13,2008-06-09T00:00:00+00:00,2008-06-09T00:00:00+00:00,2010-11-04T00:00:00+00:00,2010-11-04T00:00:00+00:00,1.0,None,Low,False,[],[]
109,4565,Tengen Toppa Gurren Lagann Movie 2: Lagann-hen,https://myanimelist.net/anime/4565/Tengen_Topp...,https://myanimelist.net/anime/4565/Tengen_Topp...,https://cdn.myanimelist.net/images/anime/12/19...,Movie,2,10,1,1,2,None,None,False,False,False,False,None,R+,2010-10-04T00:00:00+00:00,2010-10-04T00:00:00+00:00,2012-06-01T00:00:00+00:00,2012-06-01T00:00:00+00:00,1.0,None,Low,False,[],[]
110,5947,Tengen Toppa Gurren Lagann Movie Zenyasai: Vir...,https://myanimelist.net/anime/5947/Tengen_Topp...,https://myanimelist.net/anime/5947/Tengen_Topp...,https://cdn.myanimelist.net/images/anime/13/22...,OVA,2,8,1,1,2,None,None,False,False,False,False,None,PG-13,2008-12-01T00:00:00+00:00,2008-12-01T00:00:00+00:00,2010-09-01T00:00:00+00:00,2010-09-01T00:00:00+00:00,1.0,None,Low,False,[],[]
111,6548,Tengen Toppa Gurren Lagann: Kirameki★Yoko Box ...,https://myanimelist.net/anime/6548/Tengen_Topp...,https://myanimelist.net/anime/6548/Tengen_Topp...,https://cdn.myanimelist.net/images/anime/9/249...,Music,2,8,1,1,2,None,None,False,False,False,False,None,PG-13,2011-03-05T00:00:00+00:00,2011-03-05T00:00:00+00:00,None,None,NaN,None,Low,False,[],[]
112,3352,Tengen Toppa Gurren Lagann: Mitee Mono wa Mite...,https://myanimelist.net/anime/3352/Tengen_Topp...,https://myanimelist.net/anime/3352/Tengen_Topp...,https://cdn.myanimelist.net/images/anime/9/196...,Special,2,8,0,1,2,None,None,False,False,False,False,None,PG-13,2010-02-02T00:00:00+00:00,2010-02-02T00:00:00+00:00,2007-04-12T00:00:00+00:00,2007-04-12T00:00:00+00:00,1.0,None,Low,False,[],[]
113,4705,Tengen Toppa Gurren Lagann: Parallel Works,https://myanimelist.net/anime/4705/Tengen_Topp...,https://myanimelist.net/anime/4705/Tengen_Topp...,https://cdn.myanimelist.net/images/anime/5/109...,Music,2,9,8,8,2,None,None,False,False,False,False,None,PG-13,2009-03-06T00:00:00+00:00,2009-02-09T00:00:00+00:00,2009-03-06T00:00:00+00:00,2009-02-09T00:00:00+00:00,92.0,None,Low,False,[],[]
114,8348,Tengen Toppa Gurren Lagann: Parallel Works 2,https://myanimelist.net/anime/8348/Tengen_Topp...,https://myanimelist.net/anime/8348/Tengen_Topp...,https://cdn.myanimelist.net/images/anime/13/23...,Music,2,8,7,7,2,None,None,False,False,False,False,None,PG-13,2012-02-05T00:00:00+00:00,2012-02-05T00:00:00+00:00,2010-04-06T00:00:00+00:00,2010-04-06T00:00:00+00:00,1.0,None,Low,False,[],[]


In [250]:
orionite_completed_lst = list(orionite_completed["mal_id"])

2001 in orionite_completed_lst

True

In [251]:
user = get_uid("orionite")

user_orionite = anime.reset_index()[["anime_id"]]

user_orionite["est"] = user_orionite["anime_id"].apply(lambda item: algo.predict(user, item)[3] if item not in orionite_completed_lst else 0)

user_orionite.sort_values("est", ascending=False).head(10)["anime_id"].apply(lambda x: anime.loc[x])

,title,image_url
7483,Redline,https://myanimelist.cdn-dena.com/images/anime/...
4687,Detroit Metal City,https://myanimelist.cdn-dena.com/images/anime/...
3677,Prison School,https://myanimelist.cdn-dena.com/images/anime/...
10598,Daicon Opening Animations,https://myanimelist.cdn-dena.com/images/anime/...
10810,JoJo no Kimyou na Bouken (TV),https://myanimelist.cdn-dena.com/images/anime/...
13232,Nihon Animator Mihonichi,https://myanimelist.cdn-dena.com/images/anime/...
9889,Great Teacher Onizuka,https://myanimelist.cdn-dena.com/images/anime/...
6687,Mahou Shoujo Lyrical Nanoha: The Movie 2nd A&#...,https://myanimelist.cdn-dena.com/images/anime/...
5005,Hellsing Ultimate,https://myanimelist.cdn-dena.com/images/anime/...
9880,Uchuu Senkan Yamato 2199,https://myanimelist.cdn-dena.com/images/anime/...


In [178]:
user_orionite

,anime_id,est
0,11013,7.910471
1,2104,9.330880
2,5262,7.880556
3,721,7.892112
4,12365,8.488308
...,...,...
14473,26089,7.376596
14474,21525,7.705882
14475,37897,7.706382
14476,34193,7.706382


In [254]:
# user factors (n_users, n_factors)
algo.pu.shape

(271620, 100)

In [253]:
# item factors (n_items, n_factors)
algo.qi.shape

(14054, 100)

In [226]:
# user biases (n_users)
algo.bu[0]

0.25164323312979164

In [227]:
# item biases (n_items)
algo.bi[0]

-0.056219577209310805

{}